##**3 Assignment**

다음 미완성 코드를 활용해 좀 더 긴 문장을 학습해보자

* Sample sentences
  * "if you want to build a ship, don't drum up people together to "
  * "collect wood and don't assign them tasks and work, but rather "
  * "teach them to long for the endless immensity of the sea."
* Training data sentence
  * Shape: (N, S, E)
* Hidden
  * Shape: (N, S, E *2)
* Output
  * Shape: (N, S, E)

In [66]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from difflib import SequenceMatcher

# for reproducibility
torch.manual_seed(100)

# Dictionary
sample_sentence_1 = "if you want to build a ship, don't drum up people together to "
sample_sentence_2 = "collect wood and don't assign them tasks and work, but rather "
sample_sentence_3 = "teach them to long for the endless immensity of the sea."
sample_sentence = sample_sentence_1 + sample_sentence_2 + sample_sentence_3
char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

# Dataset setting
input_batch = []
target_batch = []
input_batch1 = []

x_data = [dic[c] for c in sample_sentence[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [dic[c] for c in sample_sentence[1:]]

input_batch1.append(x_one_hot)
X1 = torch.FloatTensor(np.array(input_batch))

num = 0
N = 0
for _ in range(160):
    mini_batch_x = []
    mini_batch_y = []
    num = N
    for _ in range(unit_sequence_length):
        mini_batch_x.append(x_one_hot[num])
        mini_batch_y.append(y_data[num])
        num += 1
    N += 1
    input_batch.append(mini_batch_x)
    target_batch.append(mini_batch_y)

X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))
print(X.shape)
print(Y.shape)

class Custom_RNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_RNN, self).__init__()
    self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers)
    self.fc = nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

class Custom_LSTM(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_LSTM, self).__init__()
    self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers)
    self.fc = nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.lstm(x)
    x = self.fc(x)
    return x

class Custom_GRU(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_GRU, self).__init__()
    self.gru = torch.nn.GRU(input_dim, hidden_dim, num_layers=layers)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.gru(x)
    x = self.fc(x)
    return x

learning_rate = 0.05
training_epochs = 100
model1 = Custom_RNN(input_size, hidden_size, output_size, 2)
model2 = Custom_LSTM(input_size, hidden_size, output_size, 2)
model3 = Custom_GRU(input_size, hidden_size, output_size, 2)

# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer1 = optim.Adam(model1.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(model2.parameters(), lr=learning_rate)
optimizer3 = optim.Adam(model3.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    optimizer3.zero_grad()

    output1 = model1(X)
    output2 = model2(X)
    output3 = model3(X)

    loss1 = criterion(output1.reshape(-1, dic_size), Y.reshape(-1))
    loss2 = criterion(output2.reshape(-1, dic_size), Y.reshape(-1))
    loss3 = criterion(output3.reshape(-1, dic_size), Y.reshape(-1))
    loss1.backward()
    loss2.backward()
    loss3.backward()
    optimizer1.step()
    optimizer2.step()
    optimizer3.step()

    if epoch % 10 == 9:
        print('RNN  ','epoch: ',epoch, 'loss: ', loss1.item())
        print('LSTM  ','epoch: ',epoch, 'loss: ', loss2.item()) 
        print('GRU  ','epoch: ',epoch, 'loss: ', loss3.item())
        print('------------------------------------------')

# result
result1 = output1.data.numpy().argmax(axis=2)
result2 = output2.data.numpy().argmax(axis=2)
result3 = output3.data.numpy().argmax(axis=2)

result_sentence1 = []
for i,  character in enumerate(result1):
  result_sentence1.append(char_set[character[0]])
  if i == len(result1) - 1:
      for sen in character:
        result_sentence1.append(char_set[sen])

result_sentence2 = []
for i,  character in enumerate(result2):
  result_sentence2.append(char_set[character[0]])
  if i == len(result2) - 1:
      for sen in character:
        result_sentence2.append(char_set[sen])
        
result_sentence3 = []
for i,  character in enumerate(result3):
  result_sentence3.append(char_set[character[0]])
  if i == len(result3) - 1:
      for sen in character:
        result_sentence3.append(char_set[sen])     
        
result_sentence1 = ''.join(result_sentence1)
result_sentence2 = ''.join(result_sentence2)
result_sentence3 = ''.join(result_sentence3)   

ratio1 = SequenceMatcher(None, result_sentence1, sample_sentence).ratio()
ratio2 = SequenceMatcher(None, result_sentence2, sample_sentence).ratio()
ratio3 = SequenceMatcher(None, result_sentence3, sample_sentence).ratio()

print('RNN Result: ',result_sentence1)
print('RNN Accuracy: ',ratio1)
print('------------------------------------------')
print('LSTM Result: ',result_sentence2)
print('LSTM Accuracy: ',ratio2) 
print('------------------------------------------')
print('GRU Result: ',result_sentence3)
print('GRU Accuracy: ',ratio3)

torch.Size([160, 20, 25])
torch.Size([160, 20])
RNN   epoch:  9 loss:  2.8800301551818848
LSTM   epoch:  9 loss:  2.8676562309265137
GRU   epoch:  9 loss:  2.1204166412353516
------------------------------------------
RNN   epoch:  19 loss:  2.810518741607666
LSTM   epoch:  19 loss:  2.840773344039917
GRU   epoch:  19 loss:  0.5478891730308533
------------------------------------------
RNN   epoch:  29 loss:  2.6897928714752197
LSTM   epoch:  29 loss:  2.7778165340423584
GRU   epoch:  29 loss:  0.0734465941786766
------------------------------------------
RNN   epoch:  39 loss:  2.6187591552734375
LSTM   epoch:  39 loss:  2.6021804809570312
GRU   epoch:  39 loss:  0.020794501528143883
------------------------------------------
RNN   epoch:  49 loss:  2.450483798980713
LSTM   epoch:  49 loss:  2.3426361083984375
GRU   epoch:  49 loss:  0.008466236293315887
------------------------------------------
RNN   epoch:  59 loss:  2.2634596824645996
LSTM   epoch:  59 loss:  1.9280929565429688
GR